## Extraer información de API Last FM

- Hay que ir a la página de Last FM y registrarse para obtener una API KEY. https://www.last.fm/api 
- Consultar la documentación de la API de Last FM para saber como hacer las consultas.https://www.last.fm/api 
- La dirección URL de la API de Last FM es: http://ws.audioscrobbler.com/2.0/

In [1]:
import requests

In [2]:
import pandas as pd

In [3]:
# Este script consulta los artistas en base a una lista que le pasemos (que traemos desde la consulta de spotify). 
# Devuelve Artista, Biography, Listeners, Playcount, Similar Artists

# API Key obtenida de last.fm
API_KEY = '9e5692d440bb47dc2917211142fce605'  # Reemplaza con tu API Key válida de Last.fm.
BASE_URL = 'http://ws.audioscrobbler.com/2.0/' # esta base siempre es la misma.

In [4]:
df_artistas = pd.read_csv('pop_artistas.csv', index_col = 0)

In [5]:
df_artistas.head()

,0
0,Ildikó Keresztes
1,Amalia
2,Elsie Bayron
3,One Ring Zero
4,Gal George Gjurin


In [6]:
type(df_artistas) 

pandas.core.frame.DataFrame

In [7]:
artist_data = df_artistas['0'].tolist()

In [8]:
type(artist_data)

list

In [9]:
len(artist_data)

612

In [10]:
data_info = []

for artist in artist_data: 
    params_info = {
        'method': 'artist.getinfo', 
        'artist': artist, 
        'api_key': API_KEY, 
        'format': 'json'
        }
    response_info = requests.get(BASE_URL, params = params_info)
    data_info.append(response_info.json())


In [11]:
len(data_info)

612

In [12]:
data_artista = []

for i in range(0, len(data_info)):
    similar_artist = []
    for sim in range(0, len(data_info[i]['artist']['similar']['artist'])): 
        similar_artist.append(data_info[i]['artist']['similar']['artist'][sim]['name'])
    similar_artist = tuple(similar_artist)
    artista ={
        'Artist': data_info[i]['artist']['name'], 
        'Biography': data_info[i]['artist'].get('bio', {}).get('summary'), 
        'Listeners': data_info[i]['artist'].get('stats', {}).get('listeners', 'Sin datos'), 
        'Playcount': data_info[i]['artist'].get('stats', {}).get('playcount', 'Sin datos'),
        'Similar': similar_artist
        }
    data_artista.append(artista)

In [13]:
len(data_artista)

612

In [14]:
data_artista[55]

{'Artist': 'Pasión Vega',
 'Biography': 'Pasión Vega lanzó su primer álbum en 2002. Después alcanzó el Disco de Oro con el disco Banderas de nadie (2003) y repitió Oro con Flaca de amor (2005) antes de publicar su CD+DVD en directo Pasión en el Maestranza (2005), grabado durante un concierto en Sevilla. Su cuarto álbum es La reina del Pay Pay (2006), con el que consiguió su tercer Disco de Oro en España, antes de editar el recopilatorio Pura Pasión (2007). Con estos álbumes, Pasión Vega ha ganado cuatro Premios de la Música y un Premio Amigo, entre otros galardones. <a href="https://www.last.fm/music/+noredirect/Pasi%C3%B3n+Vega">Read more on Last.fm</a>',
 'Listeners': '24858',
 'Playcount': '176142',
 'Similar': ('Carlos Cano',
  'Diana Navarro',
  'Rocio Jurado',
  'Falete',
  'Concha Piquer')}

In [15]:
df_lastfm_pop = pd.DataFrame(data_artista)

In [16]:
df_lastfm_pop.head()

,Artist,Biography,Listeners,Playcount,Similar
0,Ildikó Keresztes,"<a href=""https://www.last.fm/music/Ildik%C3%B...",311,4258,"(Veca Janicsák, Pál Dénes, usnk, Viktor Kiraly..."
1,Amalia,"Amalia is a mononym of, at least, three artist...",8073,59523,"(Opolopo, SoulParlor, Vindahl, Opolopo & Amali..."
2,Elsie Bayron,"<a href=""https://www.last.fm/music/Elsie+Bayr...",285,1028,"(Orquesta Gran Casino, Rafael Medina, Los Xey,..."
3,One Ring Zero,One Ring Zero is Michael Hearst and Joshua Cam...,4332,44075,"(Die Hard Cafe, The Gertrudes, Dream Brother, ..."
4,Gal George Gjurin,"<a href=""https://www.last.fm/music/Gal+George...",46,425,"(Severa Gjurin, Kokosy, MRFY, Jan Plestenjak, ..."


In [17]:
df_lastfm_pop.to_csv('pop_lastfm.csv')